<a href="https://colab.research.google.com/github/BaileyDalton007/discord_rnn/blob/main/discord_rnn_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from gensim.models.word2vec import Word2Vec as w2v

from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

import csv
import pandas as pd
import numpy as np

import plotly.graph_objects as go

# Matplotlib outputs tons of warning for missing characters
import warnings
warnings.filterwarnings('ignore')

from google.colab import files

In [ ]:
# Upload training csv's
uploaded = files.upload()

In [ ]:
file_names = list(uploaded.keys())

In [ ]:
from os import linesep
# Create master array for messages from all files
msg_array = []

for file in file_names:
  with open(file, 'r', encoding='utf8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')

    for row in csv_reader:
      msg_array.append(row[0].lower())

# Tokenize msg_array
msg_array = [sub.split() for sub in msg_array]

In [ ]:
# If model already trained and saved, load it.
model_save_file = files.upload()
model = w2v.load('word2vec.model')

In [ ]:
# Train word2vec model, may take a bit
model = w2v(msg_array, min_count=2, size = 5, window = 5)

print(model)

In [ ]:
model.save('word2vec.model')
files.download('word2vec.model')

In [7]:
vocab = list(model.wv.vocab)

In [ ]:
print(model.wv.__getitem__('cars'))

In [ ]:
# This is horrendous please don't

X = model.wv.__getitem__(model.wv.vocab)
pca = PCA(n_components=2)

result = pca.fit_transform(X)

# Create a scatter plot of the projection
plt.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)

for i, word in enumerate(words):
   plt.annotate(word, xy=(result[i, 0], result[i, 1]))

plt.show()

In [ ]:
# Pass the embeddings to PCA
X = model.wv.__getitem__(model.wv.vocab)
pca = PCA(n_components=2)
result = pca.fit_transform(X)

# Create df from the pca results
pca_df = pd.DataFrame(result, columns = ['x','y'])

# Add the words for the hover effect
pca_df['word'] = words
pca_df.head()